### Setting up Training Data ###

In [1]:
import numpy as np
import h5py
import csv
import matplotlib.pyplot as plt
with h5py.File("data.hdf5", 'r') as f:
    train_data = f['train'][()]
    test_data = f['test'][()]
labels = open('train_labels.csv')
labels_reader = csv.reader(labels)
classes = ["FM","OQPSK","BPSK","8PSK","AM-SSB-SC","4ASK","16PSK","AM-DSB-SC","QPSK","OOK"]
train_labels_data = []
for row in labels_reader:
    if row[0] != 'Id':
        train_labels_data.append(classes.index(row[1]))

train_labels_data = np.asarray(train_labels_data)
print(train_data.shape)
print(len(train_labels_data))
print(test_data.shape)


    

(30000, 1024, 2)
30000
(20000, 1024, 2)


### Visualizing Data  ###

In [2]:
# for j in range(0,50):
#     I = []
#     Q = []
#     for i in range(0,1024):
#         I.append(train_data[j][i][0])
#         Q.append(train_data[j][i][1])
#     plt.figure(j)
#     plt.scatter(I,Q)



### Setting up Convolutional Neural Network Architecture ###

In [17]:
from __future__ import print_function
import torch
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import torchvision
from torchvision import datasets, transforms

# converting training images into torch format
train_x = train_data.reshape(30000, 1, 1024, 2)
train_x  = torch.from_numpy(train_x)

# train_fft_x = fft_train_2d.reshape(30000, 1, 1024, 2)
# train_fft_x  = torch.from_numpy(train_fft_x)

# converting test images into torch format
test_x = test_data.reshape(20000, 1, 1024, 2)
test_x  = torch.from_numpy(test_x)

# converting the target into torch format
train_y = train_labels_data.astype(int)
train_y = torch.from_numpy(train_y)


# split into batches
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1,shuffle=True)

train_x1, train_x2, train_y1,train_y2 = train_test_split(train_x, train_y, test_size = 0.50,shuffle=True)
train_x3, train_x4, train_y3,train_y4 = train_test_split(train_x1, train_y1, test_size = 0.50,shuffle=True)
train_x5, train_x6, train_y5,train_y6 = train_test_split(train_x2, train_y2, test_size = 0.7,shuffle=True)
# batches
train = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(train, batch_size=120, shuffle=True)

# shape of training data
print(train_x.shape)
print(train_y.shape)

import torch.nn as nn

class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 5, kernel_size=(3,3), stride=1, padding=7)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
        self.conv2 = nn.Conv2d(5, 5, kernel_size=5, stride=1, padding=2)
        self.batchnorm = nn.BatchNorm2d(num_features=5)
        self.fc1 = nn.Linear(3885, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, len(classes))

    def forward(self, x):
        x = self.pool(F.selu(self.batchnorm(self.conv1(x))))
        x = self.pool(F.selu(self.batchnorm(self.conv2(x))))
        x = x.view(-1, 3885)
        x = F.selu((self.fc1(x)))
        x = F.selu((self.fc2(x)))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 256, kernel_size=(3,1), stride=1)
        self.conv2 = nn.Conv2d(256, 80, kernel_size=(3,2), stride=1)
        self.conv3 = nn.Conv2d(80, 80, kernel_size=(3,1), stride=1)
        self.fc1 = nn.Linear(81280, 128)
        self.fc2 = nn.Linear(128, len(classes))

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.selu(self.conv3(x))
        x = x.view(-1, 81280)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x, dim=1)
        
cnn = Net2()
print(cnn)

it = iter(trainloader)

#calculate dimensions
x = torch.randn(3,1,1024,2)
conv1 = nn.Conv2d(1, 256, kernel_size=(3,1), stride=1)
y=conv1(x)
print(y.shape)
conv2 = nn.Conv2d(256, 80, kernel_size=(3,2), stride=1)
y=conv2(y)
print(y.shape)
conv3 = nn.Conv2d(80, 80, kernel_size=(3,1), stride=1)
y=conv3(y)
y=conv3(y)
print(y.shape)
#65152

torch.Size([27000, 1, 1024, 2])
torch.Size([27000])
Net2(
  (conv1): Conv2d(1, 256, kernel_size=(3, 1), stride=(1, 1))
  (conv2): Conv2d(256, 80, kernel_size=(3, 2), stride=(1, 1))
  (conv3): Conv2d(80, 80, kernel_size=(3, 1), stride=(1, 1))
  (fc1): Linear(in_features=81280, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
torch.Size([3, 256, 1022, 2])
torch.Size([3, 80, 1020, 1])
torch.Size([3, 80, 1016, 1])


# Set up training protocol ###

In [18]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters(),lr=0.00006)
    error = nn.CrossEntropyLoss()
    EPOCHS = 10000
    model.train()
    for epoch in range(EPOCHS):
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()
            predicted = torch.max(output.data, 1)[1] 
            print("Accuracy:")
            print(((predicted == var_y_batch).sum()).double()/120)
            print("Loss:")
            print(loss)
            print("Epoch:")
            print(epoch)
            print("Batch:")
            print(batch_idx)
            
            
            


In [19]:

fit(cnn,trainloader)



Accuracy:
tensor(0.0750, dtype=torch.float64)
Loss:
tensor(2.3047, grad_fn=<NllLossBackward>)
Epoch:
0
Batch:
0
Accuracy:
tensor(0.1167, dtype=torch.float64)
Loss:
tensor(2.3024, grad_fn=<NllLossBackward>)
Epoch:
0
Batch:
1
Accuracy:
tensor(0.0833, dtype=torch.float64)
Loss:
tensor(2.3163, grad_fn=<NllLossBackward>)
Epoch:
0
Batch:
2
Accuracy:
tensor(0.1333, dtype=torch.float64)
Loss:
tensor(2.2996, grad_fn=<NllLossBackward>)
Epoch:
0
Batch:
3
Accuracy:
tensor(0.1667, dtype=torch.float64)
Loss:
tensor(2.2922, grad_fn=<NllLossBackward>)
Epoch:
0
Batch:
4
Accuracy:
tensor(0.1500, dtype=torch.float64)
Loss:
tensor(2.2927, grad_fn=<NllLossBackward>)
Epoch:
0
Batch:
5
Accuracy:
tensor(0.1250, dtype=torch.float64)
Loss:
tensor(2.2958, grad_fn=<NllLossBackward>)
Epoch:
0
Batch:
6
Accuracy:
tensor(0.1000, dtype=torch.float64)
Loss:
tensor(2.3223, grad_fn=<NllLossBackward>)
Epoch:
0
Batch:
7
Accuracy:
tensor(0.1333, dtype=torch.float64)
Loss:
tensor(2.3084, grad_fn=<NllLossBackward>)
Epoch:
0
B

KeyboardInterrupt: 

### Predictions ###

In [17]:
with torch.no_grad():
        output = cnn(test_x)


In [18]:
prob = output.tolist()
predictions = np.argmax(prob, axis=1)

pred_label = []
for i in predictions:
    pred_label.append(classes[i])
from numpy import asarray
from numpy import savetxt
import pandas as pd
# define id
id = np.arange(0,20000)
labels=np.asarray(pred_label)
labels.reshape(20000,1)
print(labels.shape)
d = {'Category':labels}
df = pd.DataFrame(data=d)
df.to_csv('submission.csv')

(20000,)
